This is my very first entry in the Playground Series, so this could be a bit rusty at first, but I'm looking to improve as much as possible. Either way, we begin by exploring the data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")

In [ ]:
train.columns

In [ ]:
train.head()

Let's just check for any missing values before doing data analysis. 

In [ ]:
missing_values = train.isnull().sum()
print(missing_values[missing_values > 0])

No imputation needed. Let's create a diagram taking a look at the frequency of the different classes.

In [ ]:
#Begin by getting counts for the different classes.
train.sort_values('target')
values = train['target'].value_counts()
values = values.sort_index()
values

In [ ]:
values.plot(kind='bar')

We have most of the data coming from classes 6 and 8, with class 5 having the fewest entries.

With that, let's split the data into training and validation sets.

In [ ]:
from sklearn.model_selection import train_test_split
y = train['target']              
train.drop(['target'], axis=1, inplace=True)
X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.8, test_size=0.2, random_state = 0)

**Baseline XGBoost Model**

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
model_1 = XGBClassifier()
model_1.fit(X_train, y_train)
predictions_1 = model_1.predict_proba(X_valid)

In [ ]:
log_1 = log_loss(y_valid, predictions_1)
log_1

A good start. However, we can do better by employing Optuna. 

**Optuna**

In [ ]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
#Credit here: https://medium.com/subex-ai-labs/efficient-hyperparameter-optimization-for-xgboost-model-using-optuna-3ee9a02566b1
def objective(trial: Trial, X, y) -> float:
    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
                'max_depth':trial.suggest_int('max_depth', 2, 25),
                'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
                'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
                'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
                'gamma':trial.suggest_int('gamma', 0, 5),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
                'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
                'nthread' : -1
            }
    
    model = XGBClassifier(**param)
    
    return cross_val_score(model, X, y, cv=3).mean()

In [ ]:
#Uncomment this if you want to actually see the process.

#X = train
#study = optuna.create_study(direction='maximize',sampler=TPESampler())
#study.optimize(lambda trial: objective(trial, X, y), n_trials = 7)
#print(study.best_trial)

Trial 7 = Best. Value = 0.2757101639146051. 
Parameters: 
{'n_estimators': 318, 'max_depth': 4, 'reg_alpha': 2, 'reg_lambda': 1, 'min_child_weight': 4, 'gamma': 0, 'learning_rate': 0.007170944988728181, 'colsample_bytree': 0.37}

In [ ]:
params = {'n_estimators': 318, 
          'max_depth': 4, 
          'reg_alpha': 2, 
          'reg_lambda': 1, 
          'min_child_weight': 4, 
          'gamma': 0, 
          'learning_rate': 0.007170944988728181, 
          'colsample_bytree': 0.37}
model_2 = XGBClassifier(params)
model_2.fit(X_train, y_train)
predictions_2 = model_2.predict_proba(X_valid)
log_2 = log_loss(y_valid, predictions_2)
log_2

Progress made. 

In [ ]:
model_fin = model_2
prediction_fin = model_fin.predict_proba(test)
prediction_fin

In [ ]:
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4','Class_5','Class_6', 'Class_7', 'Class_8', 'Class_9']] = prediction_fin
sample_submission.to_csv('predictions.csv',index=False)
sample_submission